In [1]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
#I IMPORTED EVERY LIBRARY THAT I WILL NEED LATER
print("Done")

Done


In [2]:
latitude=43.65
altitude=-79.35
#I GOOGLED THE COORDINATES OF TORONTO, AND THEN PROCEDED TO CREATE A MAP IN  FOLIUM
tmap = folium.Map(location=[latitude, altitude], zoom_start=10)
tmap
#THE MAP ISN'T VISIBLE IN GITHUB, CHECK OUT THE README FILE

I need to make a quick clarification before we continue, Wikipedia didn't provided me with the right data so I had to look for an actual dataset of Canada ZIP codes, all of them, the dataset contained over 10 000 rows and some useless columns for this exercise.
So I worked with that dataset in PostgreSQL to make it smaller and easier to work with(feel free to use it, it is on my Github repository), as you will see the Dataset I created its on point, no errors(102 rows, the same amount of zip codes in Toronto)

In [3]:
source = 'C:/Users/edxo/Downloads/export(2).csv'
df = pd.read_csv(source, error_bad_lines= False, warn_bad_lines=False)
df.head()

,place_name,latitude,longitude
0,East Toronto (The Danforth East),43.6872,-79.3368
1,Central Toronto (Davisville North),43.7135,-79.3887
2,East Toronto (The Beaches),43.6784,-79.2941
3,East Toronto (India Bazaar / The Beaches West),43.6693,-79.3155
4,Downtown Toronto (Central Bay Street),43.6564,-79.3860


In [4]:
#HERE I WILL CREATE A MAP WITH THE DATASET I PREVIOUSLY MENTIONED
for lat, lng, neighbourhood in zip(df['latitude'], df['longitude'], df['place_name']):
    label = neighbourhood
    label = folium.Popup(label)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(tmap)
tmap
#THE MAP ISN'T VISIBLE IN GITHUB, CHECK OUT THE README FILE

In [5]:
#I WILL APPLY KMEAN TO CLUSTER THE NEIGHBOURHOODS IN FIVE CLUSTERS AND TRAIN THE MODEL
k=5
toronto_clustering = df.drop(['place_name'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df.insert(0, 'Cluster Labels', kmeans.labels_)
df

,Cluster Labels,place_name,latitude,longitude
0,2,East Toronto (The Danforth East),43.6872,-79.3368
1,4,Central Toronto (Davisville North),43.7135,-79.3887
2,2,East Toronto (The Beaches),43.6784,-79.2941
3,2,East Toronto (India Bazaar / The Beaches West),43.6693,-79.3155
4,1,Downtown Toronto (Central Bay Street),43.6564,-79.3860
...,...,...,...,...
97,0,Etobicoke (The Kingsway / Montgomery Road / Ol...,43.6518,-79.5076
98,0,Etobicoke (Westmount),43.6949,-79.5323
99,0,Etobicoke (Eringate / Bloordale Gardens / Old ...,43.6437,-79.5767
100,0,Etobicoke (Kingsview Village / St. Phillips / ...,43.6898,-79.5582


In [6]:
#AND FINALLY I WILL CREATE A MAP WERE YOU CAN VISUALISE THE CLUSTERS IN THE MAP
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df['latitude'], df['longitude'], df['place_name'], df['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters
#THE MAP ISN'T VISIBLE IN GITHUB, CHECK OUT THE README FILE